In [1]:
# Parameters
num_cores = 14
mem = 700000
config = "configs.config_Reddit"


In [2]:
#!/usr/bin/env python
# coding: utf-8

if 'config' not in locals():
    config = 'configs.config_SE_test_local'
#     config = 'configs.config_Reddit_test_local'

In [3]:
### initialization ###

from _imports import *
# from _utils import *

print('config file is set to {}'.format(config))

util=importlib.import_module('_utils')
importlib.reload(util)

c=importlib.import_module(config)
importlib.reload(c)

get_ipython().run_line_magic('matplotlib', 'inline')

# param_setup(sys.argv[1:], c)
util.param_setup_ipython(globals(), c)

config file is set to configs.config_Reddit
num_cores is set to 14
mem is set to 700000


In [4]:
### loading optimal model db and tags file ###

if c.tags is not None:
    df_tags = pd.read_csv('{}{}_0_tags.csv'.format(c.directory['save'], c.project_name))
    c.tags_dict = dict(zip(df_tags['key'], df_tags['val']))

dbfile = open('{}{}_db_optimal_model-{}-{}.pickle'.format(c.directory['save'],
                                                          c.project_name,
                                                          c.optimal_LDA_iteration_threshold,
                                                          c.num_topics
                                                         ), 'rb') 
db=pickle.load(dbfile)                      
dbfile.close()

optimal_model=db['model']
optimal_model_corpus=db['optimal_model_corpus']
c.num_topics=db['num_topics']
df_text=db['df_text']
data_lemmatized=df_text['txt_lemmatized']

pprint('Number of topics = {}'.format(c.num_topics))
pprint('Number of docs = {}'.format(len(data_lemmatized)))

'Number of topics = 62'
'Number of docs = 2225657'


In [5]:
### showing topics & saving to pickle file ###

topics = list()
df_topics=pd.DataFrame()

for i in range(c.num_topics):
    wp = optimal_model.show_topic(i)
    topics.append(", ".join([util.decode_tag(word, c) for word, prop in wp]))
    df_topics=pd.concat([df_topics, 
                         pd.DataFrame([util.decode_tag(word, c) for word, prop in wp], 
                                      columns=['Topic {}'.format(i+1)])
                        ], 
                        axis=1)

df_topics_saving= pd.DataFrame(topics, columns=['top keywords'])
df_topics_saving.index=range(1, c.num_topics+1)
df_topics_saving.to_csv('{}{}_8_{}_topics.csv'.format(c.directory['save'], 
                                                      c.project_name, 
                                                      c.num_topics
                                                     ), 
                   encoding='utf-8', 
                  )

# df_topics.index=range(1, 11)
# c.display_df(df_topics.T)
# df_topics

try:
    c.os.remove(c.directory['save']+'{}_db_viz_data_{}.pickle'.format(c.project_name, c.num_topics))
except:
    pass

dbfile = open(c.directory['save']+'{}_db_viz_data_{}.pickle'.format(c.project_name, c.num_topics), 'ab')
db=dict()

db['topic_weights']=optimal_model.show_topics(num_topics=c.num_topics, formatted=False)

def str_to_list(txt):
    return [term.strip() for term in re.split(',', txt)]

def str_to_list_df(df):
    return df.apply(str_to_list)

data_flat = util.parallelize_df(data_lemmatized, str_to_list_df, c)
data_flat = [term for terms in data_flat for term in terms]

counter = Counter(data_flat)
out = []
for i, topic in db['topic_weights']:
    for word, weight in topic:
        out.append([util.decode_tag(word, c), i , weight, counter[word]])
db['topic_hist']=out

util.display_df(df_topics)

parallel: 14 partitions with 14 cores for str_to_list_df


,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10,Topic 11,Topic 12,Topic 13,Topic 14,Topic 15,Topic 16,Topic 17,Topic 18,Topic 19,Topic 20,Topic 21,Topic 22,Topic 23,Topic 24,Topic 25,Topic 26,Topic 27,Topic 28,Topic 29,Topic 30,Topic 31,Topic 32,Topic 33,Topic 34,Topic 35,Topic 36,Topic 37,Topic 38,Topic 39,Topic 40,Topic 41,Topic 42,Topic 43,Topic 44,Topic 45,Topic 46,Topic 47,Topic 48,Topic 49,Topic 50,Topic 51,Topic 52,Topic 53,Topic 54,Topic 55,Topic 56,Topic 57,Topic 58,Topic 59,Topic 60,Topic 61,Topic 62
0,data,learn,gpu,pay,peopl,googl,energi,www,differ,data,eat,flight,time,day,word,data,wiki,war,distribut,temperatur,video,drug,news,game,pay,vote,white,car,research,school,peopl,work,peopl,phone,feel,imag,gun,kid,movi,citi,dog,number,read,cour,govern,comment,woman,fuck,make,model,project,game,problem,drink,data,bitcoin,data,imag,countri,data,peopl,compani
1,valu,paper,tensorflow,money,thing,page,water,org,group,sourc,food,plane,peopl,time,languag,python,index,militari,function,year,post,peopl,trump,win,tax,state,black,drive,ai,student,religion,job,countri,internet,time,train,peopl,child,show,live,back,data,someth,learn,peopl,reddit,man,peopl,point,data,work,play,time,buy,job,money,graph,link,american,question,make,market
2,column,model,run,hous,realli,site,power,html,thing,tool,fat,fli,thing,hour,text,r,visual,countri,valu,earth,realli,smoke,articl,play,money,elect,peopl,road,paper,colleg,muslim,peopl,high,appl,realli,layer,kill,parent,episod,state,hand,averag,good,machin learn,state,post,male,shit,fact,train,code,team,work,beer,work,valu,color,tri,europ,answer,person,product
3,cluster,train,model,year,lot,user,build,pdf,correl,visual,calori,ship,happen,year,english,code,data,world,point,warm,interest,vaccin,sourc,move,cost,trump,race,driver,human,educ,christian,pay,popul,app,make,network,shoot,peopl,watch,area,guy,rate,start,book,law,subreddit,femal,make,argument,predict,interest,sport,thing,food,data scienc,currenc,line,index,world,inform,thing,busi
4,code,algorithm,comput,live,good,search,electr,jpg,peopl,www,weight,travel,chang,week,sentenc,languag,post,russia,probabl,climat,make,doctor,medium,number,spend,parti,racist,time,work,class,religi,make,world,netflix,good,input,crime,famili,good,peopl,wear,high,point,math,system,link,gender,guy,claim,featur,find,player,make,good,scientist,buy,make,probabl,immigr,give,someon,make
5,file,neural network,work,cost,make,data,produc,png,point,creat,diet,space,make,work,write,tool,plea,fight,model,human,watch,medic,report,time,govern,presid,minor,mile,ml,year,believ,time,rate,pay,life,model,polic,age,season,place,time,year,tri,ml,power,r,sex,call,peopl,set,idea,time,system,store,experi,bank,show,currentthread,popul,survey,feel,money
6,import,network,code,buy,actual,facebook,nuclear,c,factor,excel,bodi,airport,problem,month,differ,work,comment,china,sampl,chang,find,caus,polit,chanc,insur,republican,asian,vehicl,field,univer,church,compani,poor,googl,thing,output,murder,live,song,popul,make,popul,understand,good,social,thread,girl,stupid,actual,test,tri,watch,good,cheap,compani,dollar,map,c,canada,make,tri,price
7,c,gener,train,save,someth,websit,coal,imag,gener,make,make,time,year,everi,speak,sql,wikiflair,american,number,planet,cool,die,lie,random,healthcar,candid,group,truck,machin learn,degre,god,money,low,iphon,love,gener,suicid,year,music,map,put,show,find,start,make,user,attract,thing,wrong,dataset,good,good,type,make,program,peopl,chart,post,peopl,respon,realli,sell
8,function,optim,librari,rent,mani,track,oil,file,level,map,obe,airlin,point,spend,charact,learn,postremov,power,weight,water,youtub,cancer,stori,bet,state,democrat,racism,traffic,scienc,teacher,group,worker,incom,watch,friend,weight,death,life,film,counti,cat,total,work,python,countri,rule,guy,reddit,comment,good,write,win,realli,price,skill,price,visual,originsourc,uk,point,care,profit
9,return,work,kera,make,work,analyt,cost,googl,effect,tableau,day,speed,start,minut,letter,d

In [6]:
### Topic normalization with minimum topic cutoff ###

def normalize(row):
    try:
        # prop=[x[1] if x[1]>c.topic_cutoff_threshold else 0 for x in row]
        prop=[]
        for item in row:
            if item is None:
                util.count_sync(1000)
                prop.append(0)
            else:
                prop.append(item[1] if item[1] > c.topic_cutoff_threshold else 0)
        # prop = [x[1] if x is not None and x[1] > c.topic_cutoff_threshold else 0 for x in row]
        if np.sum(prop)==0:
            return pd.Series(prop)
        return pd.Series(prop/np.sum(prop), index=range(c.num_topics))
    except:
        raise ValueError('row = {}'.format(row))

def normalize_df(df):
    return df.apply(normalize, axis=1)


df_optimal_model = pd.DataFrame(optimal_model_corpus)
# c.init_parallel(True)
# df_optimal_model_normalized=df_optimal_model.parallel_apply(normalize, axis=1)
df_optimal_model_normalized=util.parallelize_df(df_optimal_model, normalize_df, c)

print('None value = {}'.format(util.counter_sync))
util.display_df(df_optimal_model_normalized.head(10))

parallel: 14 partitions with 14 cores for normalize_df


None value = 1


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61
0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.265107,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.734893,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.340023,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.319749,0.000000,0.000000,0.0,0.340228,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.843221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.156779,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.154423,0.689157,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.15642,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.189241,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.596866,0.000000,0.000000,0.0,0.000000,0.0,0.213893,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.522439,0.0,0.477561,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.237128,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.609549,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.153323,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.130756,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.576009,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.293235,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [7]:
### find dominant topic for each document ###

def find_dominant_topic(row):
    row = sorted(enumerate(row), key=lambda x: (x[1]), reverse=True)
    topic_num=row[0][0]
    prop_topic=row[0][1]
    return pd.Series([int(topic_num),
                                     round(prop_topic,4), 
                                     topics[topic_num]]
                                   )
def find_dominant_topic_df(df):
    return df.apply(find_dominant_topic, axis=1)

util.init_parallel(True, c)
df_topic_sents_keywords = df_optimal_model_normalized.parallel_apply(find_dominant_topic, axis=1)
# df_topic_sents_keywords=util.parallelize_df(df_optimal_model_normalized, find_dominant_topic_df, c)

col_selected=['txt_orig', 'date', 'id', 'group']
col_renamed=['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords', 'Text', 'Date', 'Id', 'Group']

if list(c.template['files'].values())[0]['cols']['category'] is not None:
    col_selected.append('category')
    col_renamed.append('category')
    
df_topic_sents_keywords = pd.concat([df_topic_sents_keywords, 
                                            df_text[col_selected]], 
                                           ignore_index=True, 
                                           axis=1)

util.save_df_to_sql(df_topic_sents_keywords,
          '{}{}_sql_docs_dominant_topic.sqlite'.format(c.directory['save'],c.project_name),
          'docs'
         )

df_topic_sents_keywords.columns=col_renamed
util.display_df(df_topic_sents_keywords.head(10))

New pandarallel memory created - Size: 50000 MB
Pandarallel will run on 14 workers


,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text,Date,Id,Group,category
0,1,1.0000,"learn, paper, model, train, algorithm, neural ...",Measuring human performance on standard image ...,2015-01-01 00:29:40,2qzibe,submissions,MachineLearning
1,58,0.7349,"countri, american, europ, world, immigr, popul...",European economy guide: Taking Europe’s pulse,2015-01-01 03:52:22,2qzv35,submissions,dataisbeautiful
2,45,0.3402,"comment, reddit, post, subreddit, link, r, thr...",Popularity of visual forms in DataIsBeautiful ...,2015-01-01 05:23:25,2qzzaa,submissions,dataisbeautiful
3,40,1.0000,"dog, back, hand, guy, wear, time, make, put, c...",JOE GROOMING - DAILY SHAMPOO ~ best shampoo fo...,2015-01-01 07:16:15,2r048c,submissions,dataisbeautiful
4,19,0.8432,"temperatur, year, earth, warm, climat, human, ...",Los Angeles Traffic Accident Rate in Rainy vs ...,2015-01-01 10:25:46,2r0e7b,submissions,dataisbeautiful
5,43,0.6892,"cour, learn, machin learn, book, math, ml, goo...",How did you get good at ML? I feel like practi...,2015-01-01 11:02:17,2r0gu6,submissions,MachineLearning
6,41,0.5969,"number, data, averag, rate, high, year, popul,...",Last year some guy posted how many times he po...,2015-01-01 11:33:15,2r0jax,submissions,dataisbeautiful
7,47,0.5224,"fuck, peopl, shit, make, guy, call, stupid, th...",How to lie with Visualization: A humorous less...,2015-01-01 12:02:42,2r0lux,submissions,datascience
8,15,0.6095,"data, python, r, code, languag, tool, work, sq...",Recommended Web Analytics Software? a company ...,2015-01-01 13:21:21,2r0tbc,submissions,analytics
9,18,0.5760,"distribut, function, valu, point, probabl, mod...",Learning Information Geometry I have recently ...,2015-01-01 13:30:09,2r0u4u,submissions,MachineLearning


In [8]:
### Group top K sentences under each topic from original weights + saving ###


def find_dominant_topic_for_original_weight(row):
    # c.print_asynch(row[0])
    topic_num=row[0][0]
    prop_topic=row[0][1]

    for item in row[1:]:
#         c.print_asynch('item in for {} - {} - {}'.format(item, item[0], item[1]))
        if item is None or item[0] is None or item[1] is None:
            util.count_sync(1000)
        elif item[1]> prop_topic:
            topic_num=item[0]
            prop_topic=item[1]

    # row = sorted(enumerate(row), key=lambda x: (x[1][1] if x[1][1] is not None else 0), reverse=True)
    # topic_num=row[0][1][0]
    # prop_topic=row[0][1][1]

    return pd.Series([int(topic_num),
                                     round(prop_topic,4), 
                                     topics[topic_num]]
                                   )

def find_dominant_topic_for_original_weight_df(df):
    return df.apply(find_dominant_topic_for_original_weight, axis=1)

# c.init_parallel(True)
# df_optimal_original = df_optimal_model.parallel_apply(find_dominant_topic_for_original_weight, axis=1)
df_optimal_original = util.parallelize_df(df_optimal_model, find_dominant_topic_for_original_weight_df, c)

util.print_asynch('{} number of None in optimal model df!'.format(util.counter_sync))

col_selected=['all_text', 'txt_orig', 'keywords', 'id', 'group']
col_renamed=['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords',
                                     'all_text', 'txt_orig', 'txt_lemmatized', 'id', 'group']

if list(c.template['files'].values())[0]['cols']['category'] is not None:
    col_selected.append('category')
    col_renamed.append('category')
    
df_optimal_original = pd.concat([df_optimal_original,
                                 df_text[col_selected]],
                                ignore_index=True, 
                                axis=1
                               )
df_optimal_original.columns=col_renamed
sent_topics_outdf_grpd = df_optimal_original.groupby('Dominant_Topic')

# sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')


sent_topics_sorteddf_mallet=pd.DataFrame()
for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet,
                                             grp.sort_values(['Perc_Contribution'],
                                                             ascending=[0]).head(c.num_sent)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# sent_topics_sorteddf_mallet.drop(['Date'], inplace=True, axis=1)

# Format
# sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", 
#                                        'all_text', 'txt_orig', 'keywords']

# Show
sent_topics_sorteddf_mallet.to_csv('{}{}_9_top_{}_samples_by_dominant_topics.csv'.format(c.directory['save'], 
                                                                                         c.project_name,
                                                                                         c.num_sent
                                                                                        ),
                                   index=False)

util.display_df(sent_topics_sorteddf_mallet.head(10))

parallel: 14 partitions with 14 cores for find_dominant_topic_for_original_weight_df


1 number of None in optimal model df!


,Dominant_Topic,Perc_Contribution,Topic_Keywords,all_text,txt_orig,txt_lemmatized,id,group,category
0,0,0.9723,"data, valu, column, cluster, code, file, impor...",Dict iterations are now order-of-assignment?? ...,Dict iterations are now order-of-assignment?? ...,"dict, iter, order, assign, think, use, notat, ...",dtedg4n,comments,MachineLearning
1,0,0.9695,"data, valu, column, cluster, code, file, impor...",How do I import a large (+TB) csv dataframe in...,How do I import a large (+TB) csv dataframe in...,"import, larg, csv, datafram, hdf, object, key,...",56elm6,submissions,bigdata
2,0,0.9693,"data, valu, column, cluster, code, file, impor...",How to massage this data into a dataframe (e.g...,How to massage this data into a dataframe (e.g...,"massag, data, datafram, eg, panda, datafram, f...",4tyio1,submissions,datascience
3,0,0.9688,"data, valu, column, cluster, code, file, impor...",Tensorflow feed_dict Error I have the followin...,Tensorflow feed_dict Error I have the followin...,"tensorflow, feeddict, error, follow, code, pri...",81vufz,submissions,learnmachinelearning
4,0,0.9642,"data, valu, column, cluster, code, file, impor...",How to avoid repetations in the next_batch fun...,How to avoid repetations in the next_batch fun...,"avoid, repet, nextbatch, function, tensorflow,...",95x1wh,submissions,MLQuestions
5,0,0.9620,"data, valu, column, cluster, code, file, impor...",My program. clc; %Gen CWorld and CImage points...,My program. \nclc; \n%Gen CWorld and CImage po...,"program, clc, gen, cworld, cimag, point, fov, ...",czb4xh2,comments,MachineLearning
6,0,0.9613,"data, valu, column, cluster, code, file, impor...",true and if u check the next file ex1.multi I ...,true and if u check the next file ex1.multi I ...,"true, u, check, next, file, multi, figur, late...",e1xyrhp,comments,learnmachinelearning
7,0,0.9600,"data, valu, column, cluster, code, file, impor...","okay, so I just posted the wrong code. I'm sor...","okay, so I just posted the wrong code. I'm sor...","wrong, code, sorri, anoth, question, decod, on...",d7jbjtw,comments,MachineLearning
8,0,0.9586,"data, valu, column, cluster, code, file, impor...",You could more simply use the cache function i...,You could more simply use the cache function i...,"simpli, cach, function, python, doc, python, o...",dzof8j4,comments,dataisbeautiful
9,0,0.9570,"data, valu, column, cluster, code, file, impor...","So I now have: def makePoint( lon:Double, lat:...",So I now have:\n\n def makePoint( lon:Doubl...,"def, makepoint, lon, doubl, lat, doubl, point,...",d5qxz41,comments,datascience


In [9]:
### Number of Documents for Each Topic - original ###

topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = np.round(100 * topic_counts/topic_counts.sum(), 2)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([
    pd.Series(range(1, c.num_topics+1)), 
    pd.Series(topics), 
    topic_counts, 
    topic_contribution
], axis=1)

# Change Column names
df_dominant_topics.columns = ['Topic_Num', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

df_dominant_topics.to_csv('{}{}_10_dominant_topics_dist_original.csv'.format(c.directory['save'], 
                                                                            c.project_name
                                                                           ),
                         index=False)
# Show
util.display_df(df_dominant_topics)




,Topic_Num,Topic_Keywords,Num_Documents,Perc_Documents
0,1,"data, valu, column, cluster, code, file, impor...",8190,0.37
1,2,"learn, paper, model, train, algorithm, neural ...",28665,1.29
2,3,"gpu, tensorflow, run, model, comput, work, cod...",31255,1.40
3,4,"pay, money, hous, year, live, cost, buy, save,...",40608,1.82
4,5,"peopl, thing, realli, lot, good, make, actual,...",51134,2.30
5,6,"googl, page, site, user, search, data, faceboo...",41271,1.85
6,7,"energi, water, power, build, electr, produc, n...",26645,1.20
7,8,"www, org, html, pdf, jpg, png, c, imag, file, ...",17058,0.77
8,9,"differ, group, thing, correl, peopl, point, fa...",20568,0.92
9,10,"data, sourc, tool, visual, www, creat, excel, ...",29012,1.30


In [10]:
### Number of Documents for Each Topic - normalized ###

topic_weights=df_optimal_model_normalized.apply(lambda col: np.sum(col), axis=0)

# Percentage of Documents for Each Topic
topic_contribution = np.round(100 * pd.DataFrame(topic_weights/np.sum(topic_weights)), 2)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics_cleaned = pd.concat([pd.Series(range(1, c.num_topics+1)), 
                                       pd.Series(topics), 
                                       pd.DataFrame(topic_weights), 
                                       topic_contribution
                                      ], 
                                      axis=1)

# Change Column names
df_dominant_topics_cleaned.columns = ['Topic_Num', 'Topic_Keywords', 'Weight_Documents', 'Perc_Documents']
df_dominant_topics_cleaned.to_csv('{}{}_10_dominant_topics_dist_normalized.csv'.format(c.directory['save'], 
                                                                            c.project_name
                                                                           ),
                         index=False)

# Show
util.display_df(df_dominant_topics_cleaned)

,Topic_Num,Topic_Keywords,Weight_Documents,Perc_Documents
0,1,"data, valu, column, cluster, code, file, impor...",7209.359679,0.32
1,2,"learn, paper, model, train, algorithm, neural ...",26789.270539,1.20
2,3,"gpu, tensorflow, run, model, comput, work, cod...",29168.288276,1.31
3,4,"pay, money, hous, year, live, cost, buy, save,...",38871.378446,1.75
4,5,"peopl, thing, realli, lot, good, make, actual,...",67528.214137,3.04
5,6,"googl, page, site, user, search, data, faceboo...",39974.193409,1.80
6,7,"energi, water, power, build, electr, produc, n...",24896.486930,1.12
7,8,"www, org, html, pdf, jpg, png, c, imag, file, ...",18800.892482,0.85
8,9,"differ, group, thing, correl, peopl, point, fa...",25158.948276,1.13
9,10,"data, sourc, tool, visual, www, creat, excel, ...",27358.690989,1.23


In [11]:
### removing docs less than the threshold for visualiztion ###

# Get topic weights
topic_weights = []
# for i, row_list in enumerate(lda_model[corpus]):
#     topic_weights.append([w for i, w in row_list[0]])

# for i, row_list in enumerate(optimal_model[corpus]):
for i, row_list in enumerate(optimal_model_corpus):
    topic_weights.append([w for i, w in row_list])
    
# Array of topic weights    
arr = pd.DataFrame(topic_weights).fillna(0).values

# Keep the well separated points (optional)
idx = np.amax(arr, axis=1) > c.TSNE_threshold
df_text_TSNE =df_text[idx]
arr = arr[idx]
util.pprint('Number of points to be disaplyed = {}'.format(len(arr)))



'Number of points to be disaplyed = 37185'


In [12]:
### TSNE dimensionality reduction ###

# tSNE Dimension Reduction
tsne_model = TSNE(n_components=2,
                  verbose=1, 
                  random_state=0, 
                  angle=.99, 
                  init='pca'
                 )
tsne_lda = tsne_model.fit_transform(arr)

db['tsne_lda'] = tsne_lda
db['df_text_TSNE'] = df_text_TSNE
db['arr'] = arr

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 37185 samples in 0.109s...


[t-SNE] Computed neighbors for 37185 samples in 25.298s...
[t-SNE] Computed conditional probabilities for sample 1000 / 37185


[t-SNE] Computed conditional probabilities for sample 2000 / 37185
[t-SNE] Computed conditional probabilities for sample 3000 / 37185
[t-SNE] Computed conditional probabilities for sample 4000 / 37185
[t-SNE] Computed conditional probabilities for sample 5000 / 37185
[t-SNE] Computed conditional probabilities for sample 6000 / 37185


[t-SNE] Computed conditional probabilities for sample 7000 / 37185
[t-SNE] Computed conditional probabilities for sample 8000 / 37185
[t-SNE] Computed conditional probabilities for sample 9000 / 37185
[t-SNE] Computed conditional probabilities for sample 10000 / 37185
[t-SNE] Computed conditional probabilities for sample 11000 / 37185
[t-SNE] Computed conditional probabilities for sample 12000 / 37185


[t-SNE] Computed conditional probabilities for sample 13000 / 37185
[t-SNE] Computed conditional probabilities for sample 14000 / 37185
[t-SNE] Computed conditional probabilities for sample 15000 / 37185
[t-SNE] Computed conditional probabilities for sample 16000 / 37185
[t-SNE] Computed conditional probabilities for sample 17000 / 37185
[t-SNE] Computed conditional probabilities for sample 18000 / 37185


...

[t-SNE] Computed conditional probabilities for sample 27000 / 37185
[t-SNE] Computed conditional probabilities for sample 28000 / 37185
[t-SNE] Computed conditional probabilities for sample 29000 / 37185


[t-SNE] Computed conditional probabilities for sample 30000 / 37185
[t-SNE] Computed conditional probabilities for sample 31000 / 37185
[t-SNE] Computed conditional probabilities for sample 32000 / 37185
[t-SNE] Computed conditional probabilities for sample 33000 / 37185
[t-SNE] Computed conditional probabilities for sample 34000 / 37185


[t-SNE] Computed conditional probabilities for sample 35000 / 37185
[t-SNE] Computed conditional probabilities for sample 36000 / 37185
[t-SNE] Computed conditional probabilities for sample 37000 / 37185
[t-SNE] Computed conditional probabilities for sample 37185 / 37185
[t-SNE] Mean sigma: 0.000000


[t-SNE] KL divergence after 250 iterations with early exaggeration: 69.636879


[t-SNE] KL divergence after 1000 iterations: 1.003152


In [13]:
### collecting data for temporal graphs ###

def timestamp_to_date(timestamp):
    return datetime.fromtimestamp(timestamp)

def timestamp_to_date_df(df):
    return df.apply(timestamp_to_date)

def str_date_to_date(date):
    try:
#         return pd.to_datetime(df_text['date'], format=c.template['files'][df_name]['formats']['date'])
        return pd.to_datetime(date)
    except ValueError as e:
        util.print_asynch(date)
        raise Failed('Had issue processing {} \n {}'.format(date, e))

def str_date_to_date_df(df):
    return df.apply(str_date_to_date)


### temporal graph data - count ###

df_name='Count'
df_dominant_topic_date = df_topic_sents_keywords[['Dominant_Topic', 'Date']]

# c.init_parallel(False)
# date = c.pd.DataFrame(df_dominant_topic_date['Date'].parallel_apply(timestamp_to_date), 
# date = pd.DataFrame(util.parallelize_df(df_dominant_topic_date['Date'], timestamp_to_date_df, c), columns=['Date'])
date = pd.DataFrame(util.parallelize_df(df_dominant_topic_date['Date'], str_date_to_date_df, c), columns=['Date'])

df_topic_date = pd.concat([date,df_dominant_topic_date['Dominant_Topic']],axis=1)
df_topic_date.index=df_topic_date['Date']


### temporal graph data - proportion ###

df_name='Proportion'
df_doc_topic_weight = pd.concat([df_optimal_model_normalized, pd.DataFrame(df_text['date'])], axis=1)

# util.init_parallel(False)
# df_doc_topic_weight['Date'] = df_topic_sents_keywords['Date'].parallel_apply(timestamp_to_date)
# df_doc_topic_weight['Date'] = util.parallelize_df(df_topic_sents_keywords['Date'], timestamp_to_date_df, c)
df_doc_topic_weight['Date'] = util.parallelize_df(df_topic_sents_keywords['Date'], str_date_to_date_df, c)
df_doc_topic_weight.index=df_doc_topic_weight['Date']

df_name='Count'
dfs_trend=dict()
for freq in ['D', 'W', 'M']:
    dfs_trend[df_name]=pd.DataFrame(columns=['Date', 
                                               df_name, 
                                               'Topic'
                                              ])
    for i in range(c.num_topics):
        df_topic = df_topic_date.loc[df_topic_date['Dominant_Topic']==i]
        g=df_topic.groupby(pd.Grouper(freq=freq))
        result = g.count()['Date']
        result = pd.DataFrame({'Date': result.index, df_name: result.values, 'Topic':i})    
        dfs_trend[df_name] = pd.concat([dfs_trend[df_name], result], axis=0, sort=True)
    
    db['{}-{}'.format(df_name, freq)]=dfs_trend[df_name]

df_name='Proportion'

for freq in ['D', 'W', 'M']:
    dfs_trend[df_name]=pd.DataFrame(columns=['Date',
                                               df_name, 
                                               'Topic'
                                              ])
    for i in range(c.num_topics):
        df_topic = df_doc_topic_weight[i]
        g=df_topic.groupby(pd.Grouper(freq=freq))
        result = g.sum()/g.count()
#         result = g.sum()
        result = pd.DataFrame({'Date': result.index, df_name: result.values, 'Topic':i})    
        dfs_trend[df_name] = pd.concat([dfs_trend[df_name], result], sort=True)
    db['{}-{}'.format(df_name, freq)]=dfs_trend[df_name]

pickle.dump(db, dbfile)
dbfile.close()

parallel: 14 partitions with 14 cores for str_date_to_date_df


parallel: 14 partitions with 14 cores for str_date_to_date_df
